In [ ]:
# Added date features

fr_holidays = holidays.France()

df["datetime"] = df["date"]
df["date"] = df["datetime"].dt.date
df["hour"] = df["datetime"].dt.hour
df["weekday"] = df["datetime"].dt.weekday
df["daymonth"] = df["datetime"].dt.strftime('%d') + "_" + df["datetime"].dt.month.astype(str)
df["IsHoliday"] = df["datetime"].dt.date.apply(lambda x: x in fr_holidays)



In [ ]:
# technical days

data["date_truncated"] = data["date"].dt.floor("D")

grouped_data = (
    data.groupby(["counter_id", "date_truncated"])
    ["log_bike_count"].sum()
    .to_frame()
    .reset_index()
    .query("log_bike_count == 0")
    [["counter_id", "date_truncated"]]
)
grouped_data["counter_id"].drop_duplicates()



filtered_data = (
    grouped_data
    .merge(data, on=["counter_id", "date_truncated"], how="right", indicator=True)
    .query("_merge == 'right_only'")
    .drop(columns=["_merge"])
)
print(f"Eliminate {data.shape[0] - filtered_data.shape[0]} data points out of {data.shape[0]}")

In [ ]:
# covid

expanded_data = data

date_ranges = [
    ("2020-10-30", "2020-12-15"),
    ("2021-03-20", "2021-05-19"),
]

expanded_data["Covid-19"] = 0 
for start, end in date_ranges:
    expanded_data["Covid-19"] |= expanded_data["date"].between(start, end).astype(int)

expanded_data.head()
